In [ ]:
#!pip install langchain

In [1]:
from langchain import SQLDatabase
from langchain.llms import VertexAI
from langchain_experimental.sql import SQLDatabaseChain
import os

In [2]:
from google.cloud.sql.connector import Connector
import sqlalchemy
import pymysql


PROJECT_ID="jared-playground"
REGION="us-central1"
INSTANCE_NAME="gen-ai-text-to-mysql"

# initialize parameters
INSTANCE_CONNECTION_NAME = f"{PROJECT_ID}:{REGION}:{INSTANCE_NAME}" # i.e demo-project:us-central1:demo-instance
print(f"Your instance connection name is: {INSTANCE_CONNECTION_NAME}")
DB_USER = "root"
DB_NAME = "classicmodels"

# initialize Connector object
connector = Connector()

# function to return the database connection object
def getconn():
    conn = connector.connect(
        INSTANCE_CONNECTION_NAME,
        "pymysql",
        user=DB_USER,
        db=DB_NAME
    )
    return conn

# create connection pool with 'creator' argument to our connection object function
# pool = sqlalchemy.create_engine(
#     "mysql+pymysql://",
#     creator=getconn,
# )

Your instance connection name is: jared-playground:us-central1:gen-ai-text-to-mysql


In [3]:
db = SQLDatabase.from_uri(
    "mysql+pymysql://",
    engine_args = {'creator': getconn},
)

In [5]:
#print(db.table_info)
print(os.environ["OPENAI_API_KEY"])

KeyError: 'OPENAI_API_KEY'

In [6]:
print(os.environ)

environ({'CONDA_PROMPT_MODIFIER': '(base) ', 'USER': 'jupyter', 'JUPYTER_DEPS_PATH': '/opt/deeplearning/jupyter', 'GCSFUSE_METADATA_IMAGE_TYPE': 'DLVM', 'SHLVL': '0', 'HOME': '/home/jupyter', 'CONDA_SHLVL': '1', 'SRC_PATH': '/opt/deeplearning/src', 'RESTRICTION_TYPE_FILE_PATH': '/opt/deeplearning/restriction', 'PROXY_URL': '6b67ddb8326c7a55-dot-us-central1.notebooks.googleusercontent.com', 'GRPC_FORK_SUPPORT_ENABLED': '0', 'FRAMEWORK_FILE_PATH': '/opt/deeplearning/metadata/framework', '_CE_M': '', 'BINARIES_PATH': '/opt/deeplearning/binaries', 'TITLE_FILE_PATH': '/opt/deeplearning/metadata/title', 'TMPDIR': '/var/tmp', 'VERSION_FILE_PATH': '/opt/deeplearning/metadata/version', 'LOGNAME': 'jupyter', 'MOUNTS_ON_ALL_CONTAINERS': '/usr/local/share/ca-certificates,/etc/ssl/certs,/etc/ca-certificates/update.d', 'OPAL_PREFIX': '/usr', 'JOURNAL_STREAM': '8:16404', 'OS_NAME': 'DEBIAN_11', '_': '/opt/conda/bin/jupyter', 'WORKSPACE_PATH': '/opt/deeplearning/workspace', '_CE_CONDA': '', 'PATH': '/

In [7]:
llm = VertexAI(model_name="code-bison@001", temperature=0.2, verbose=True)

In [8]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [9]:
PROMPT = """ 
Given an input question, create a syntactically correct query to run with no additional text in the answer.
Only provide the SQL query in the answer. Do not incluce SQLQuery: or SQL: in the answer. Here is an example:
Example Question: How many invoices are there?
Example Solution: SELECT COUNT(*) FROM Invoice;

The question: {question}
"""

In [10]:
question = "What were sales for the year 2005 by state filtered by customers based in USA?"
db_chain.run(PROMPT.format(question=question))



> Entering new SQLDatabaseChain chain...
 
Given an input question, create a syntactically correct query to run with no additional text in the answer.
Only provide the SQL query in the answer. Do not incluce SQLQuery: or SQL: in the answer. Here is an example:
Example Question: How many invoices are there?
Example Solution: SELECT COUNT(*) FROM Invoice;

The question: What were sales for the year 2005 by state filtered by customers based in USA?

SQLQuery:SELECT SUM(amount) FROM orders WHERE orderDate BETWEEN '2005-01-01' AND '2005-12-31' AND customerNumber IN (SELECT customerNumber FROM customers WHERE country = "USA")

OperationalError: (pymysql.err.OperationalError) (1054, "Unknown column 'amount' in 'field list'")
[SQL: SELECT SUM(amount) FROM orders WHERE orderDate BETWEEN '2005-01-01' AND '2005-12-31' AND customerNumber IN (SELECT customerNumber FROM customers WHERE country = "USA")]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [34]:
# print(db.table_info)
db_chain.run("How many orders were placed?")



> Entering new SQLDatabaseChain chain...
How many orders were placed?
SQLQuery:```sql
SELECT COUNT(*) FROM orders;
```

ProgrammingError: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '```sql\nSELECT COUNT(*) FROM orders;\n```' at line 1")
[SQL: ```sql
SELECT COUNT(*) FROM orders;
```]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [ ]:
from langchain.prompts.prompt import PromptTemplate

_DEFAULT_TEMPLATE = """Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Be sure to put a semi-colon after query.
Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:

{table_info}

If someone asks for the table foobar, they really mean the employee table.

Question: {input}"""
PROMPT = PromptTemplate(
    input_variables=["input", "table_info", "dialect"], template=_DEFAULT_TEMPLATE
)
PROMPT.format(input="How many orders were placed in the year 2005?", table_info="orders", dialect="MySQL 8")

In [ ]:
# db_chain = SQLDatabaseChain.from_llm(llm, db, prompt=PROMPT, verbose=True)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [ ]:
db_chain.run("How many orders were placed?")

In [10]:
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [12]:
from langchain.agents import create_sql_agent
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True
)

In [14]:
agent_executor.run("Describe the orders table")



> Entering new AgentExecutor chain...
Question: Describe the orders table
Thought: I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables. 
Action: sql_db_list_tables
Action Input: 
Observation: customers, employees, offices, orderdetails, orders, payments, productlines, products
Thought:Question: Describe the orders table
Thought: I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables. 
Action: sql_tb_schema
Action Input: orders
Observation: sql_tb_schema is not a valid tool, try another one.
Thought:Question: Describe the orders table
Thought: I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables. 
Action: sql_tb_schema
Action Input: orders
Observation: sql_tb_schema is not a valid tool, try another one.
Thought:

OutputParserException: Could not parse LLM output: `Question: Describe the orders table
Error: Unknown tool 'sql_tb_schema'. Please try one of: ['sql_db_query', 'sql_db_schema', 'sql_db_list_tables', 'sql_db_query_checker'].`

In [ ]:
import os

import sqlalchemy


pool = sqlalchemy.create_engine(
    # Equivalent URL:
    # "mysql+pymysql://root@/classicmodels?unix_socket=/cloudsql/jared-playground:gen-ai-text-to-mysql"
    sqlalchemy.engine.url.URL.create(
        drivername="mysql+pymysql",
        username='root',
        database='classicmodels',
        query={"unix_socket": '/cloudsql/jared-playground:us-central1:gen-ai-text-to-mysql'}
    ),
    # ...
)

print(pool)

In [ ]:
URL = sqlalchemy.engine.url.URL.create(
        drivername="mysql+pymysql",
        username='root',
        database='classicmodels',
        host='34.122.217.224',
        # port='3306',
)
print(URL)

In [ ]:
DB_USER = "root"
DB_NAME = "classicmodels"
DB_ADDRESS = "34.122.217.224"
SOCKET = "mysql+pymysql://root@/classicmodels?unix_socket=/cloudsql/jared-playground:gen-ai-text-to-mysql"

db = SQLDatabase.from_uri(URL)
# db = SQLDatabase.from_uri(f"mysql+pymysql://{DB_USER}:root@{DB_ADDRESS}/{DB_NAME}")
# llm = VertexAI(model_name="code-bison")

In [ ]:
DB_USER = "root"
DB_NAME = "classicmodels"
DB_ADDRESS = "34.122.217.224"
SOCKET = "mysql+pymysql://root:root@/classicmodels?unix_socket=/cloudsql/jared-playground:gen-ai-text-to-mysql"

db = SQLDatabase.from_uri(SOCKET)